# 7.8.1 KoNLPy를 사용한 영화 리뷰 분석

- 한글로 된 영화 리뷰 감성 분석

  > KoNLPY+Countvectorize를 사용한 감성 분석

  > 네이버 영화 리뷰 데이터세트 github: github.com/e9t/nsmc/

    >> ratings_train.txt,ratings_test.txt를  data folder에 다운로드

In [1]:
# tweepy와 konlpy 설치
!pip install -q tweepy==3.10 konlpy


In [2]:
import konlpy
import pandas as pd
import numpy as np


In [3]:
konlpy.__version__


'0.6.0'

데이터 파일을 읽어 리뷰 텍스트와 점수를 text_train, y_train 변수에 저장합니다. 데이터 파일의 내용은 번호, 텍스트, 레이블이 탭으로 구분되어 한 라인에 한개의 데이터 샘플이 들어 있습니다.

In [5]:
df_train = pd.read_csv('data/ratings_train.txt', delimiter='\t', keep_default_na=False)

df_train.head(n=3)

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0


In [6]:
text_train, y_train = df_train['document'].values, df_train['label'].values

In [7]:
type(text_train)

numpy.ndarray

In [8]:
print(text_train[:5])

['아 더빙.. 진짜 짜증나네요 목소리' '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'
 '너무재밓었다그래서보는것을추천한다' '교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정'
 '사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다']


같은 방식으로 테스트 데이터를 읽습니다.

In [9]:
df_test = pd.read_csv('data/ratings_test.txt', delimiter='\t', keep_default_na=False)
text_test = df_test['document'].values
y_test = df_test['label'].values

훈련 데이터와 테스트 데이터의 크기를 확인합니다.

In [ ]:
len(text_train), np.bincount(y_train) # label 0,1의 갯수

(150000, array([75173, 74827]))

In [8]:
len(text_test), np.bincount(y_test)

(50000, array([24827, 25173]))

#### Okt

- 피클링(Pickling)이란?

  > 파이썬 객체를 그대로 파일이나 바이트 스트림으로 저장(직렬화) 하는 과정.

  > 나중에 다시 불러올 때는 언피클링(unpickling) → 저장된 바이트 스트림을 다시 원래의 파이썬 객체로 복원.

KoNLPy 0.4.5 버전부터 `Twitter` 클래스가 `Okt` 클래스로 바뀌었습니다. [open-korean-text](https://github.com/open-korean-text/open-korean-text) 프로젝트는 [twitter-korean-text](https://github.com/twitter/twitter-korean-text) 프로젝트의 공식 포크입니다.

피클링을 위해 \_\_setstate__, \_\_getstate__ 메서드를 추가하여 Okt 클래스를 감쌉니다.

- Okt 클래스는 무엇을 하나?

  > KoNLPy의 대표적인 한국어 형태소 분석기.

  > 원래 이름은 Twitter였는데, 0.4.5 버전부터 Okt(Open Korean Text)로 이름이 변경.

  > 내부적으로는 open-korean-text 라이브러리를 사용.

  > 하는 일:

    >> 한글 문장을 **형태소 단위(작은 의미 단위)**로 쪼개기

    >> 각 형태소의 품사 태깅(명사, 동사, 형용사 등)

    >> 토큰화, 어간 추출, 정규화 같은 기본 전처리 제공

In [ ]:
from konlpy.tag import Okt
okt = Okt()
# morphs() : 형태소 단위 토큰화 < morph [형태]의 뜻, morpheme: 형태소
# pos() : 형태소 + 품사 태깅
# nouns() : 명사만 추출

print(okt.morphs("커널이 자꾸 죽어서 속상해요."))
# ['커널', '이', '자꾸', '죽', '어서', '속상', '하', '아요', '.']

print(okt.pos("커널이 자꾸 죽어서 속상해요."))
# [('커널', 'Noun'), ('이', 'Josa'), ('자꾸', 'Adverb'), ('죽', 'Verb'),
#  ('어서', 'Eomi'), ('속상', 'Adjective'), ('하', 'Verb'), ('아요', 'Eomi'), ('.', 'Punctuation')]

print(okt.nouns("커널이 자꾸 죽어서 속상해요."))
# ['커널']


['커널', '이', '자꾸', '죽어서', '속상해요', '.']
[('커널', 'Noun'), ('이', 'Josa'), ('자꾸', 'Noun'), ('죽어서', 'Verb'), ('속상해요', 'Adjective'), ('.', 'Punctuation')]
['커널', '자꾸']


In [13]:
from konlpy.tag import Okt

class PicklableOkt(Okt):

    def __init__(self, *args):
        self.args = args
        Okt.__init__(self, *args)

    def __setstate__(self, state):
        self.__init__(*state['args'])

    def __getstate__(self):
        return {'args': self.args}

okt = PicklableOkt()

- tfidfvectorizer__min_df: 단어가 너무 드물게 나오면 무시.

  > 3 → 3개 문서 이상에서 등장해야 어휘에 포함

  > 5, 7도 같은 의미. → 희귀 단어 제거, 과적합 방지.

- tfidfvectorizer__ngram_range: n-그램 범위.

  > (1,1): 유니그램(단어 하나)

  > (1,2): 유니그램 + 바이그램(두 단어 연속도 포함)

  > (1,3): 유니그램 + 바이그램 + 트라이그램까지 포함

- logisticregression__C: 규제 강도 (역수).

  > 0.1 → 규제가 강해서 단순한 모델

  > 1 → 보통 수준

  > 10 → 규제가 약해져서 더 복잡한 모델 허용

- TfidfVectorizer(tokenizer=okt.morphs)

  >텍스트 → 수치 벡터 변환.

  > okt.morphs: KoNLPy의 Okt 형태소 분석기로 문장을 형태소 단위로 토큰화.

    >> 예: "커널이 죽어서 속상하다" → ['커널', '이', '죽', '어서', '속상', '하다']

  > TfidfVectorizer는 이렇게 뽑힌 형태소를 이용해 단어-문서 행렬을 만들고, TF-IDF 가중치를 계산.

    >> TF = 단어 빈도

    >> IDF = 드문 단어일수록 더 큰 가중치

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

param_grid = {'tfidfvectorizer__min_df': [3, 5, 7],
              'tfidfvectorizer__ngram_range': [(1, 1), (1, 2), (1, 3)],
              'logisticregression__C': [0.1, 1, 10]}
pipe = make_pipeline(TfidfVectorizer(tokenizer=okt.morphs),
                     LogisticRegression())
grid = GridSearchCV(pipe, param_grid, n_jobs=-1)

# 그리드 서치를 수행합니다
grid.fit(text_train[:1000], y_train[:1000])
print("최상의 교차 검증 점수: {:.3f}".format(grid.best_score_))
print("최적의 교차 검증 파라미터: ", grid.best_params_)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


최상의 크로스 밸리데이션 점수: 0.718
최적의 크로스 밸리데이션 파라미터:  {'logisticregression__C': 1, 'tfidfvectorizer__min_df': 3, 'tfidfvectorizer__ngram_range': (1, 3)}


In [ ]:
tfidfvectorizer = grid.best_estimator_.named_steps["tfidfvectorizer"]
X_test = tfidfvectorizer.transform(text_test[:1000])
logisticregression = grid.best_estimator_.named_steps["logisticregression"]
score = logisticregression.score(X_test, y_test[:1000])

print("테스트 세트 점수: {:.3f}".format(score))

테스트 세트 점수: 0.713
